In [ ]:
#!pip install textblob
#!pyspark --py-files my_dependencies.zip

In [ ]:
import collections
import os
import pickle
import random
import re
import glob
from warnings import filterwarnings
import json
import ast
from pyspark.sql import SparkSession 
from pyspark.sql import functions as sf
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, StringType, FloatType, ArrayType, LongType, MapType
import tqdm
import unidecode
from textblob import TextBlob
from extracao_atributos import extrair_volumeTweets, extrair_indiceInsonia, extrair_atributos_metadados_twitter, extrair_atributos_por_regex
from extracao_atributos import extrair_links_midia_mencoes_hashtags, extrair_media_polaridade_subjetividade, extrair_media_valencia_ativacao
import numpy as np 
import pandas as pd
from utilitarios import atualizarBaseSerie, get_expressao_regular, tratarValoresAusentes, abrirBaseSerie
from utilitarios import dfSpark_to_seriesPandas, carregarBaseAnewBR, carregarBaseMedicamentosAntiDepressivos

filterwarnings('ignore')

In [ ]:
def obter_files_max_tamanho(arquivos, limite_gb):
    tamanho_total = 0
    limite_bytes = limite_gb * 1024**3  # Convertendo GB para bytes
    listaFiles = []
    for arquivo in arquivos:
        tamanho_arquivo = os.path.getsize(arquivo)
        tamanho_total += tamanho_arquivo
        if tamanho_total > limite_bytes:
            return listaFiles  # Retorna o limite se excedido
        listaFiles.append(arquivo)

    return listaFiles

In [ ]:
def listar_arquivos(path, tamanho_min_mb, tamanho_max_mb):
    tamanho_min_bytes = tamanho_min_mb * 1048576  # Convertendo de MB para bytes
    tamanho_max_bytes = tamanho_max_mb * 1048576  # Convertendo de MB para bytes

    arquivos_selecionados = []
    files = tqdm.tqdm(glob.glob(path))
    for caminho_arquivo in files:
        tamanho_arquivo = os.path.getsize(caminho_arquivo)
        if tamanho_min_bytes <= tamanho_arquivo <= tamanho_max_bytes:
            arquivos_selecionados.append(caminho_arquivo)

    return arquivos_selecionados

In [ ]:
periodo = 'pandemia'
grupo = 'depressao'

path_tweets = f"datasets/{periodo}/tweets/{grupo}/"

path_caracteristicas = f"datasets/{periodo}/caracteristicas/{grupo}/volumeTweets_serieTemporal.pickle"

usuarios = list(pd.read_pickle(path_caracteristicas).keys())

files = [path_tweets + i + '_fulljson.csv' for i in usuarios if os.path.exists(path_tweets + i + '_fulljson.csv')]

intervalo = int(len(files) * 0.1)

listaUsuarios = [os.path.basename(i).replace('_fulljson.csv', '') for i in files][:intervalo]
len(listaUsuarios)
#files = glob.glob(path + '*.csv')

In [ ]:
diretorioCaracteristicas = f'datasets/{periodo}/caracteristicas/{grupo}/'
if not os.path.exists(diretorioCaracteristicas):
    os.makedirs(diretorioCaracteristicas)

In [ ]:
min_size = 10
max_size = 30
files = listar_arquivos(path + '*.csv', min_size, max_size)
len(files)

In [ ]:
usuarios_extraidos = list(abrirBaseSerie('ativacao', diretorioCaracteristicas).keys())
len(usuarios_extraidos)

In [ ]:
#files = list(set(files) - set(usuarios_extraidos))
#files = obter_files_max_tamanho(files, 1)
len(files)

In [ ]:
name_files = [os.path.basename(i) for i in files]
listaUsuarios = [i.replace('_fulljson.csv', '') for i in name_files]
len(listaUsuarios)

In [ ]:
schema = StructType([
    StructField("created_at", TimestampType(), True),  # Data de criação como TimestampType
    StructField("id", LongType(), True),  # ID do tweet como Long
    StructField("full_text", StringType(), True),  # Texto completo do tweet
    StructField("id_screen_name", LongType(), True),  # ID do nome de usuário como Long
    StructField("screen_name", StringType(), True),  # Nome de usuário
    StructField("favorite_count", IntegerType(), True),  # Contagem de favoritos como inteiro
    StructField("media", StringType(), True),  # Mídias como uma lista de URLs (strings)
    StructField("links", StringType(), True),  # Links como uma lista de URLs (strings)
    StructField("mentioned_users", StringType(), True),  # Usuários mencionados como um dicionario de usuário
    StructField("reply_count", IntegerType(), True),  # Contagem de respostas como inteiro
    StructField("hashtags", StringType(), True),  # Hashtags como uma lista de strings
    StructField("retweet_count", IntegerType(), True)  # Contagem de retweets como inteiro
])

spark = SparkSession.builder.appName('Tweets Sentiment').getOrCreate()

df = spark.read.csv(files, header=True, inferSchema=True, sep=';', encoding='utf-8', schema=schema)
df.printSchema()

In [ ]:
'''listaUsuarios = df.select("screen_name").collect()
listaUsuarios = list(set([i['screen_name'] for i in listaUsuarios]))
len(listaUsuarios)'''

In [ ]:
# UDF para converter string em lista de dicionários
def string_to_list_dict(str_dict):
    try:
        return ast.literal_eval(str_dict)
    except (ValueError, SyntaxError):
        return []
        
# UDF para converter string em lista
def string_to_list(str_list):
    try:
        # Converte a string para uma lista de strings
        return ast.literal_eval(str_list)
    except (ValueError, SyntaxError):
        return []

converter_string_to_list_dict = sf.udf(string_to_list_dict, ArrayType(MapType(StringType(), StringType())))
converter_string_to_list = sf.udf(string_to_list, ArrayType(StringType()))

In [ ]:
def pre_processamento(df, coluna):

    acentos = 'áàãâäéèêëíìîïóòõôöúùûüç'
    sem_acentos = 'aaaaaeeeeiiiiooooouuuuc'

    df = df.withColumn(coluna, sf.lower(df[coluna]))
    df = df.withColumn(coluna, sf.trim(df[coluna]))
    
    df = df.withColumn(coluna, sf.regexp_replace(coluna, "https?:\/\/.*[\r\n]*", ""))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'\[[0-9]*\]', ' '))
    # df = df.withColumn(coluna, sf.regexp_replace(coluna, r'[^\w\s]', ''))
    df = df.withColumn(coluna, sf.translate(df[coluna], acentos, sem_acentos))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'\d', ' '))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'\s+', ' '))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, "@\S+", ""))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, "\$", ""))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, "#", ""))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'https?:\/\/[\r\n],"[\r\n]"', ''))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'\<a href', ' '))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'&amp;', ''))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' '))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'<br />', ' '))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'\'', ' '))
    df = df.withColumn(coluna, sf.regexp_replace(coluna, '[^a-zà-ù ]', ' '))
    
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r"(?:[hH]+[aA]+[hH]+[aA]+)|(?:[kK]+[kK]+[kK]+)", 'haha'))  
    
    df = df.withColumn('full_text_original', sf.col(coluna))
    
    df = df.withColumn(coluna, sf.regexp_replace(coluna, r'[^\u0000-\u024F]+', ''))

    df = df.withColumn(coluna, sf.lower(df[coluna]))
    df = df.withColumn(coluna, sf.trim(df[coluna]))

    df = df.filter(sf.col(coluna) != ' ')
    df = df.filter(sf.col(coluna) != '')
    
    return df

In [ ]:
df = pre_processamento(df, 'full_text')

df = df.withColumn("mentioned_users", converter_string_to_list_dict(df["mentioned_users"]))
df = df.withColumn("media", converter_string_to_list(df["media"]))
df = df.withColumn("links", converter_string_to_list(df["links"]))
df = df.withColumn("hashtags", converter_string_to_list(df["hashtags"]))

In [ ]:
def remover_usuarios_extraidos(lista_principal, itens_para_remover):
    for item in itens_para_remover:
        if item in lista_principal:
            lista_principal.remove(item)

    return lista_principal

# Atributo Volume de Tweets e Decteção de Outliens

In [ ]:
serieTemporal_volume_tweets = {}
outliers = {}
if listaUsuarios:
    listaNovosUsuarios = remover_usuarios_extraidos(listaUsuarios, list(outliers.keys()))
    listaNovosUsuarios = remover_usuarios_extraidos(listaNovosUsuarios, list(abrirBaseSerie('volumeTweets', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Volume de Tweets do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal = extrair_volumeTweets(usuario, df_usuario)
        
        valorMaxTweetDia = serieTemporal.max(axis=0)
        somaTotalTweets = serieTemporal.sum()
        if valorMaxTweetDia > 300:
            outliers[usuario] = valorMaxTweetDia
                    
        elif somaTotalTweets < 30:
            outliers[usuario] = somaTotalTweets

        else:
            serieTemporal_volume_tweets[usuario] = serieTemporal
            #serieTemporal_volume_tweets[usuario].fillna(value=serieTemporal_volume_tweets[usuario].mean(skipna=True), inplace=True)
            atualizarBaseSerie(serieTemporal_volume_tweets, 'volumeTweets', diretorioCaracteristicas)
            serieTemporal_volume_tweets.pop(usuario, None)

    del df_temp

with open(diretorioCaracteristicas + 'outliers.pickle', 'wb') as f:
    pickle.dump(outliers, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
diretorioOutliens = f'{diretorioCaracteristicas}outliers.pickle'
if os.path.isfile(diretorioOutliens):
    outliers = pd.read_pickle(diretorioOutliens)

listaOutliers = list(outliers.keys())
df = df.filter(~df["screen_name"].isin(listaOutliers))

# Outros Atributos

In [ ]:
# Atributo 1ª Pessoa
pronomes_1pessoa = ['eu', 'nos', "mim", "comigo", "me", "moi", "migo", "mi"]
expressaoRegular_1pessoa = get_expressao_regular(pronomes_1pessoa)

# Atributo 2ª Pessoa
pronomes_2pessoa = ['tu', 'voce', 'vc', 'te', 'ti', 'consigo', 'contigo', 'tigo', 'cê']
expressaoRegular_2pessoa = get_expressao_regular(pronomes_2pessoa)

# Atributo 3ª Pessoa
pronomes_3pessoa = ['ele', 'ela', 'eles', 'elas', 'o', 'a', 'os', 'as', 'lhe', 'lhes', 'se', 'si', 'consigo', 'dela', 'dele', 'delas', 'deles', 'nego']
expressaoRegular_3pessoa = get_expressao_regular(pronomes_3pessoa)

# Atributo Caracteres Oreintais
unicodeOrientais = ['\uac00-\ud7a3', '\u3040-\u30ff', '\u4e00-\u9FFF']
expressaoRegular_caracteresOrientais = get_expressao_regular(unicodeOrientais)

# Atributo Emojis
unicodeEmojis = ['[\U0001F300-\U000E007F]']
expressaoRegular_emojis = get_expressao_regular(unicodeEmojis)

# Atributos Valencia e Ativação
baseAnewBR_df = carregarBaseAnewBR()
baseAnewBR_listaPalavras = baseAnewBR_df.index.to_list()
baseAnewBR_listaPalavras.sort()
expressaoRegular_palavrasAnewBR = get_expressao_regular(baseAnewBR_listaPalavras)
baseAnewBR_json = json.dumps(baseAnewBR_df.T.to_dict())

# Atributos Termos Depressivos
listaTermosDepressivos = baseAnewBR_df.loc[baseAnewBR_df['Valencia-Media'] <= 4].index.to_list()
listaTermosDepressivos.sort()
expressaoRegular_termosDepressivos = get_expressao_regular(listaTermosDepressivos)

# Atributos Medicamentos Anti-depressivos
baseMedicamentosAntiDepressivos = carregarBaseMedicamentosAntiDepressivos()
listaMedicamentosAntiDepressivos = baseMedicamentosAntiDepressivos['Medicamento'].to_list()
listaMedicamentosAntiDepressivos.sort()
expressaoRegular_medicamentosAntiDepressivos = get_expressao_regular(listaMedicamentosAntiDepressivos)


In [ ]:
serieTemporal_indice_insonia = {}
serieTemporal_pronomes1pessoa = {}
serieTemporal_pronomes2pessoa = {}
serieTemporal_pronomes3pessoa = {}
serieTemporal_termosDepressivos = {}
serieTemporal_medicamentosAntidepressivos = {}

serieTemporal_caracteresOrientais = {}
serieTemporal_emojis= {}
serieTemporal_curtidas = {}
serieTemporal_grafoSocial = {}
serieTemporal_links = {}
serieTemporal_midia = {}

serieTemporal_retweets = {}
serieTemporal_mencoes = {}
serieTemporal_hashtags = {}
serieTemporal_polaridade = {}
serieTemporal_subjetividade = {}

serieTemporal_valencia = {}
serieTemporal_ativacao = {}

In [ ]:
if listaUsuarios:
    serieTemporal_volume_tweets = abrirBaseSerie('volumeTweets', diretorioCaracteristicas)
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('indiceInsonia', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Indice de Insonia do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_indice_insonia[usuario] = extrair_indiceInsonia(usuario, df_usuario, serieTemporal_volume_tweets[usuario])
        atualizarBaseSerie(serieTemporal_indice_insonia, 'indiceInsonia', diretorioCaracteristicas)
        serieTemporal_indice_insonia.pop(usuario, None)
    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('pronome1Pessoa', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Pronomes de 1ª Pessoa do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_pronomes1pessoa[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_1pessoa, 'full_text')
        atualizarBaseSerie(serieTemporal_pronomes1pessoa, 'pronome1Pessoa', diretorioCaracteristicas)
        serieTemporal_pronomes1pessoa.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('pronome2Pessoa', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Pronomes de 2ª Pessoa do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_pronomes1pessoa[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_1pessoa, 'full_text')
        atualizarBaseSerie(serieTemporal_pronomes1pessoa, 'pronome2Pessoa', diretorioCaracteristicas)
        serieTemporal_pronomes1pessoa.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('pronome3Pessoa', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Pronomes de 3ª Pessoa do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_pronomes1pessoa[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_1pessoa, 'full_text')
        atualizarBaseSerie(serieTemporal_pronomes1pessoa, 'pronome3Pessoa', diretorioCaracteristicas)
        serieTemporal_pronomes1pessoa.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('termosDepressivos', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Termos Depressivos do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_termosDepressivos[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_termosDepressivos, 'full_text')
        atualizarBaseSerie(serieTemporal_termosDepressivos, 'termosDepressivos', diretorioCaracteristicas)
        serieTemporal_termosDepressivos.pop(usuario, None)
        
    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('medicamentosAntiDepressivo', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Medicamentos Anti-Depresivos do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_medicamentosAntidepressivos[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_medicamentosAntiDepressivos, 'full_text')
        atualizarBaseSerie(serieTemporal_medicamentosAntidepressivos, 'medicamentosAntiDepressivo', diretorioCaracteristicas)
        serieTemporal_medicamentosAntidepressivos.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('caracteresOrientais', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Caracteres Orientais do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_caracteresOrientais[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_caracteresOrientais, 'full_text_original')
        atualizarBaseSerie(serieTemporal_caracteresOrientais, 'caracteresOrientais', diretorioCaracteristicas)
        serieTemporal_caracteresOrientais.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(list(abrirBaseSerie('emojis', diretorioCaracteristicas).keys())))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Emojis do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_emojis[usuario] = extrair_atributos_por_regex(usuario, df_usuario, expressaoRegular_emojis, 'full_text_original')
        atualizarBaseSerie(serieTemporal_emojis, 'emojis', diretorioCaracteristicas)
        serieTemporal_emojis.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('curtidas', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'favorite_count', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Curtidas do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_curtidas[usuario] = extrair_atributos_metadados_twitter(usuario, df_usuario, 'favorite_count')
        atualizarBaseSerie(serieTemporal_curtidas, 'curtidas', diretorioCaracteristicas)
        serieTemporal_curtidas.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('grafoSocial', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'reply_count', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Grafo Social do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_grafoSocial[usuario] = extrair_atributos_metadados_twitter(usuario, df_usuario, 'reply_count')
        atualizarBaseSerie(serieTemporal_grafoSocial, 'grafoSocial', diretorioCaracteristicas)
        serieTemporal_grafoSocial.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    #listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('retweets', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'retweet_count', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Retweets do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_retweets[usuario] = extrair_atributos_metadados_twitter(usuario, df_usuario, 'retweet_count')
        atualizarBaseSerie(serieTemporal_retweets, 'retweets', diretorioCaracteristicas)
        serieTemporal_retweets.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('links', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'links', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Links do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_links[usuario] = extrair_links_midia_mencoes_hashtags(usuario, df_usuario, 'links')
        atualizarBaseSerie(serieTemporal_links, 'links', diretorioCaracteristicas)
        serieTemporal_links.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('midia', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'media', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Midia do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_midia[usuario] = extrair_links_midia_mencoes_hashtags(usuario, df_usuario, 'media')
        atualizarBaseSerie(serieTemporal_midia, 'midia', diretorioCaracteristicas)
        serieTemporal_midia.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('mencoes', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'mentioned_users', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Menções do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_mencoes[usuario] = extrair_links_midia_mencoes_hashtags(usuario, df_usuario, 'mentioned_users')
        atualizarBaseSerie(serieTemporal_mencoes, 'mencoes', diretorioCaracteristicas)
        serieTemporal_mencoes.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('hashtags', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'hashtags', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Hashtags do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_hashtags[usuario] = extrair_links_midia_mencoes_hashtags(usuario, df_usuario, 'hashtags')
        atualizarBaseSerie(serieTemporal_hashtags, 'hashtags', diretorioCaracteristicas)
        serieTemporal_hashtags.pop(usuario, None)

    del df_temp

In [ ]:
if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('subjetividade', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)

        pbar.set_description(f"Extraido Polaridade e Subjetividade do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_polaridade[usuario], serieTemporal_subjetividade[usuario] = extrair_media_polaridade_subjetividade(usuario, df_usuario)

        atualizarBaseSerie(serieTemporal_polaridade, 'polaridade', diretorioCaracteristicas)
        atualizarBaseSerie(serieTemporal_subjetividade, 'subjetividade', diretorioCaracteristicas)

        serieTemporal_polaridade.pop(usuario, None)
        serieTemporal_subjetividade.pop(usuario, None)

    del df_temp

In [ ]:
'''if listaUsuarios:
    listaUsuarios = list(set(listaUsuarios) - set(list(outliers.keys())))
    listaNovosUsuarios = list(set(listaUsuarios) - set(abrirBaseSerie('ativacao', diretorioCaracteristicas).keys()))
    pbar = tqdm.tqdm(listaNovosUsuarios)
    tamanhoUsuarios = len(listaNovosUsuarios)
    df_temp = df.select('created_at', 'full_text', 'screen_name')
    for numero, usuario in enumerate(pbar, 1):
        df_usuario = df_temp.filter(df['screen_name'] == usuario)
        
        pbar.set_description(f"Extraido Valencia e Ativação do usuario {numero} de {tamanhoUsuarios}")
        serieTemporal_valencia[usuario], serieTemporal_ativacao[usuario] = extrair_media_valencia_ativacao(usuario, df_usuario, baseAnewBR_json, expressaoRegular_palavrasAnewBR)

        atualizarBaseSerie(serieTemporal_valencia, 'valencia', diretorioCaracteristicas)
        atualizarBaseSerie(serieTemporal_ativacao, 'ativacao', diretorioCaracteristicas)

        serieTemporal_valencia.pop(usuario, None)
        serieTemporal_ativacao.pop(usuario, None)

    del df_temp'''